In [ ]:
import time

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import pandas as pd
from openpyxl import load_workbook

import configparser 
config = configparser.ConfigParser()
config.read('config.ini')
ip = config['DEFAULT']['IP']
port = config['DEFAULT']['MongoDB-Port']

from pymongo import MongoClient
client = MongoClient(ip, int(port))

import twitter

In [ ]:
CK = config['TWITTER']['CK']
CS = config['TWITTER']['CS']
ATK = config['TWITTER']['ATK']
ATS = config['TWITTER']['ATS']
Substring = config['TWITTER']['Substring']

# set to extended mode and sleep if exceed rate limit
api = twitter.Api(consumer_key=CK,
                  consumer_secret=CS,
                  access_token_key=ATK,
                  access_token_secret=ATS,
                  tweet_mode= 'extended',
                  sleep_on_rate_limit=True)

In [ ]:
db_twitter = client["Twitter"]
collections_twitter = db_twitter.collection_names()

In [ ]:
dic_collection = {}
for i in collections_twitter:
    dic_collection[i] = "{:,}".format(db_twitter[i].find({}).count())


for key in sorted(dic_collection):
    print("%s: %s" % (key, dic_collection[key]))

In [ ]:
for e in sorted(dic_collection)[::4]:
    if Substring in e:
        print("--------------\n" + e)
        c = 0
        # get the first "limit_number" records from "twitter-australia" collection
        for document in db_twitter[e].find({"truncated":True,"soda_checked_truncated_utc":{"$exists":False}}):
                c += 1
                try:
                    # get full_text using twitter api
                    full_text = api.GetStatus(document["id_str"]).full_text
                    print(str(c) + "--------------\n" + "got full_text")
                    
                    t = int(time.time())
                    # update fields(add full_text if exist and change truncated value to false)
                    db_twitter[e].update({"_id": document["_id"]}, {"$set": {"text": full_text, "truncated":False , "soda_checked_truncated_utc": t }})
                except:
                    
                    t = int(time.time())
                    db_twitter[e].update({"_id": document["_id"]}, {"$set": {"soda_checked_truncated_utc": t}})
                    print(str(c) + "--------------\n" + "pass except")

        print("--------------\n" + "Done: get full_text of " + str(c) + " truncated tweets in collection " + e)
    